In [1]:
import findspark
findspark.init('/home/nono/spark-2.4.0-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema=True,
                      sep='\t')

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed('_c0','class')

In [6]:
data = data.withColumnRenamed('_c1','text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length', length(data['text']))

In [14]:
data.show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class|text                                                                                                                                                                                                |length|
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                                                     |111   |
|ham  |Ok lar... Joking wif u oni...                                                                                                                    

In [15]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [16]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,
                                IDF,StringIndexer)

In [17]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

In [19]:
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')

In [20]:
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')

In [21]:
idf = IDF(inputCol='c_vec',outputCol='tf_idf')

In [23]:
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [24]:
from pyspark.ml.feature import VectorAssembler


In [45]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [26]:
from pyspark.ml.classification import NaiveBayes


In [46]:
nb = NaiveBayes()


In [31]:
from pyspark.ml import Pipeline

In [47]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,
                                 stop_remove,count_vec,idf,clean_up])

In [48]:
cleaner = data_prep_pipe.fit(data)

In [49]:
clean_data = cleaner.transform(data)

In [50]:
clean_data = clean_data.select('label','features')

In [51]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [52]:
training,test = clean_data.randomSplit([0.7,0.3])

In [53]:
spam_detector = nb.fit(training)

In [55]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [56]:
test_results = spam_detector.transform(test)

In [57]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-885.47119806845...|[1.0,2.1859528928...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-448.23074887071...|[1.0,7.8722711978...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1383.7842210204...|[1.0,3.1202549710...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-991.13870821150...|[1.0,2.1897301874...|       0.0|
|  0.0|(13424,[0,1,146,1...|[-251.14102595924...|[0.99303192821268...|       0.0|
|  0.0|(13424,[0,1,3657,...|[-127.81765769991...|[0.99998248140215...|       0.0|
|  0.0|(13424,[0,2,3,5,6...|[-2586.4280837344...|[1.0,3.1335314205...|       0.0|
|  0.0|(13424,[0,2,3,8,2...|[-1604.7714237705...|[1.0,1.2950283365...|       0.0|
|  0.0|(13424,[0,2,4,5,1...|[-2491.8896441076...|[1.0,1.7775750137...|       0.0|
|  0.0|(13424,[0

In [58]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [59]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)

In [60]:
print ('ACC of NB Model')

ACC of NB Model


In [61]:
print(acc)

0.9232653434946491
